# Load Packages

In [1]:

include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Utility Function

Recall: the CRRA utility function is $U(W) = \frac{W^{1-k}}{1-k}$

In [3]:
U(x,k) =  x.^(1-k)/(1-k)    #CRRA utility function, k is the risk aversion

U (generic function with 1 method)

In [4]:
W = linspace(0.8,1.2,25)


plot(W,U(W,2),color=:red,linewidth=2,legend=false)
title!("Utility function, CRRA")
xlabel!("wealth")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 -1.2 
 
 
 -1.1 
 
 
 -1.0 
 
 
 -0.9 
 
 
 Utility function, CRRA 
 
 
 wealth 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.1436,360.065 61.6924,339.949 84.2412,320.637 106.79,302.083 129.339,284.242 151.888,267.074 174.436,250.542 196.985,234.612 219.534,219.25 242.083,204.427 
 264.632,190.116 287.18,176.289 309.729,162.924 332.278,149.996 354.827,137.486 377.376,125.373 399.925,113.638 422.473,102.265 445.022,91.2359 467.571,80.5363 
 490.12,70.1513 512.669,60.0673 535.217,50.2715 557.766,40.7515 580.315,31.4961 
 "/>

# Expected Utility

Recall: if $\pi_s$ is the probability of outcome $W_s$ and there are $S$ possible outcomes, then

$\text{E}W = \sum\nolimits_{s=1}^{S} \pi_{s}W_{s}$ and 

$\text{E}U(W) = \sum\nolimits_{s=1}^{S} \pi_{s}U(W_{s})$

Concavity of the utility function means that 
$\text{E} U(W) < U(\text{E} W)$

In [5]:
EU(x,π,k) = sum(π.*U(x,k))               #expected utility

EU (generic function with 1 method)

In [6]:
W₁ = 0.85               #possible outcomes
W₂ = 1.15
π₁ = 0.5                #probabilities of outcomes 
π₂ = 1 - π₁

xx = [W₁ U(W₁,2);
      W₂ U(W₂,2)]
println("Possible wealth levels and the corresponding utility")
printmat(xx)

UtilExpW = U(sum([π₁;π₂].*[W₁;W₂]),2)
ExpUtil  = EU([W₁;W₂],[π₁;π₂],2)
printlnPs("Utility at expected value and Expected utility: ",[UtilExpW ExpUtil])

Possible wealth levels and the corresponding utility
     0.850    -1.176
     1.150    -0.870

Utility at expected value and Expected utility:     -1.000    -1.023


In [7]:
plot(W,U(W,2),color=:red,linewidth=2,label="Utility fn")
scatter!([W₁;W₂],U([W₁;W₂],2),color=:green,label="possible outcomes")
scatter!([1.0],[ExpUtil],color=:blue,marker=:rect,label="Expected utility")
plot!(ylims=(-1.3,-0.7))
title!("Utility function, CRRA")
xlabel!("wealth")
ylabel!("")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 1.2 
 
 
 -1.3 
 
 
 -1.2 
 
 
 -1.1 
 
 
 -1.0 
 
 
 -0.9 
 
 
 -0.8 
 
 
 -0.7 
 
 
 Utility function, CRRA 
 
 
 wealth 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 54.4598,332.685 75.7322,318.715 97.0047,305.304 118.277,292.419 139.55,280.029 160.822,268.107 182.095,256.627 203.367,245.564 224.639,234.896 245.912,224.603 
 267.184,214.664 288.457,205.062 309.729,195.781 331.002,186.803 352.274,178.116 373.547,169.704 394.819,161.555 416.092,153.656 437.364,145.998 458.636,138.567 
 479.909,131.355 501.181,124.353 522.454,117.55 543.726,110.939 564.999,104.511 
 "/>
 
 
 
 
 
 
 
 
 
 
 Utility fn 
 
 
 
 
 possible outcomes 
 
 
 
 
 Expected utility

# Certainty Equivalent

Recall: the certainty equivalent (here denoted $P$) is the sure (non-stochastic) value that solves 

$U(P) = \text{E}U(W)$

The code below solves for $P$ by a numerial search (actually, we could do it analytically in this case). This is applied to different values of risk aversion.

Notice that the expected wealth level is 1.

In [8]:
using Roots

In [9]:
println("risk aversion and certainly equivalent of process with equal probs of $W₁ and $W₂")
kM = [0.0001;2;5;10;25]
for k in kM
    ceq = fzero(P->EU([W₁;W₂],[π₁;π₂],k) - P^(1-k)/(1-k),[0.5,1.5])
    printlnPs([k ceq])
end

risk aversion and certainly equivalent of process with equal probs of 0.85 and 1.15
     0.000     1.000
     2.000     0.978
     5.000     0.947
    10.000     0.912
    25.000     0.875


# Portfolio Choice with One Risky Asset

Recall: the investor solves $\max\nolimits_{v}\text{E}\ln R_{p}\text{, with }R_{p}=vR_{1} + (1-v)R_{f}$

In the example below, there are two possible outcomes for $R_{1}$ (-0.2 and 0.5) with equal probabilities

In [10]:
function EUc(v,π,Re,Rf)             #expected utility, utility fn is logarithmic
    R = Re + Rf
    Rp = v*R + (1-v)*Rf             #portfolio return
    EU = sum(π .* log.(1+Rp))        #expected utility
    return EU
end

EUc (generic function with 1 method)

In [11]:
v = linspace(-1,1,101)
L = length(v)
EUv = fill(NaN,L)
for i = 1:L
    EUv[i] = EUc(v[i],[0.5;0.5],[-0.3;0.4],0.1)
end

In [12]:
    
plot(v,EUv,color=:red,linewidth=2,legend=false)
title!("Expected Utility")
xlabel!("weight on risky asset")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 Expected Utility 
 
 
 weight on risky asset 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 45.4014,360.065 50.7505,350.03 56.0996,340.202 61.4488,330.578 66.7979,321.154 72.147,311.927 77.4962,302.893 82.8453,294.049 88.1945,285.391 93.5436,276.917 
 98.8927,268.624 104.242,260.508 109.591,252.568 114.94,244.8 120.289,237.201 125.638,229.77 130.988,222.504 136.337,215.401 141.686,208.458 147.035,201.673 
 152.384,195.044 157.733,188.569 163.082,182.247 168.431,176.074 173.781,170.05 179.13,164.172 184.479,158.44 189.828,152.85 195.177,147.402 200.526,142.095 
 205.875,136.926 211.225,131.894 216.574,126.998 221.923,122.236 227.272,117.607 232.621,113.111 237.97,108.745 243.319,104.508 248.669,100.4 254.018,96.4196 
 259.367,92.5653 264.716,88.8362 270.065,85.2316 275.414,81.7503 280.763,78.3917 286.112,75.1547 291.462,72.0386 296.811,69.0426 302.16,66.166 307.509,63.4081 
 312.858,60.7681 318.207,58.2454 323.556,55.8395 328.906,53.5497 334.255,51.3755 339.604,49.3163 344.953,47.3717 350.302,45.5412 355.651,43.8243 361,42.2206 
 366.35,40.7299 371.699,39.3516 377.048,38.0855 382.397,36.9313 387.746,35.8887 393.095,34.9575 398.444,34.1374 403.793,33.4283 409.143,32.8299 414.492,32.3422 
 419.841,31.9651 425.19,31.6984 430.539,31.542 435.888,31.4961 441.237,31.5604 446.587,31.7351 451.936,32.0203 457.285,32.4158 462.634,32.9219 467.983,33.5387 
 473.332,34.2664 478.681,35.1049 484.031,36.0547 489.38,37.1159 494.729,38.2887 500.078,39.5735 505.427,40.9705 510.776,42.4801 516.125,44.1026 521.474,45.8383 
 526.824,47.6879 532.173,49.6515 537.522,51.7299 542.871,53.9233 548.22,56.2324 553.569,58.6578 558.918,61.1999 564.268,63.8595 569.617,66.6372 574.966,69.5336 
 580.315,72.5496 
 "/>

Notice that this problem can be solved by pen and paper, but it 
becomes very difficult when the number of states increase --- and even worse when there are many assets. We therefore apply a numerical optimization algorithm.

In [13]:
using Optim          #minimization routines

In [14]:
#minimize -EU()
Sol = optimize(v-> -EUc(v,[0.5;0.5],[-0.3;0.4],0.1),-1,1)   #try v values between -1 and 1
printlnPs("argmin -EUc(): ",Optim.minimizer(Sol))

println("Compare with the figure")

argmin -EUc():      0.458
Compare with the figure


# Portfolio Choice with Several Risky Assets

This optimization problem has two risky assets and a general CRRA utility function. Numerical optimization is still easy.

In [15]:
function EUe(v,π,R,Rf,γ)
    Rp = v'R + (1-sum(v))*Rf            #portfolio return, row vector (different states)
    Rp = vec(Rp)                        #turn Rp into a column vector
    EU = sum(π .* (1+Rp).^(1-γ)/(1-γ))  #expected utility
    EUNeg = -EU                         #since optimize() minimizes
    return EUNeg
end

EUe (generic function with 1 method)

In [16]:
ROutComes = [-0.03 0.08 0.20;           #2 assets, 3 states
             -0.04 0.22 0.15]           #cell[i,j] is asset i in state j
π = [1/3;1/3;1/3]
Sol = optimize(v->EUe(v,π,ROutComes,0.065,5),[-0.6;1.2])         
printlnPs("optimal weights from max EUe(): ",Optim.minimizer(Sol))

optimal weights from max EUe():     -0.725     1.316


# From Chapter on Mean-Variance Analysis

In [17]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n) 
    Σ_1  = inv(Σ)
    A    = μ'Σ_1*μ
    B    = μ'Σ_1*oneV
    C    = oneV'Σ_1*oneV
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt(w'Σ*w)
    return StdRp,w
end

function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF
    n   = length(μ)
    μe  = μ - Rf              #excess returns            
    Σ_1 = inv(Σ)
    w   = (mustar-Rf)/(μe'Σ_1*μe) .* Σ_1*μe
    StdRp = sqrt(w'Σ*w)
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

# Mean-Variance and the Telser Criterion

Let $\mu$ be a vector of expected returns and $\Sigma$ be the covariance matrix of the investible asssets.

The Telser criterion solves the problem

$\max_{v} \mu_{p} \> \text{ subject to} \>  \text{VaR}_{95\%} < 0.1$,

where $\mu_{p} = v'\mu+(1-v)R_f$ is the expected portfolio return.

If the returns are normally distributed then 

$\text{VaR}_{95\%} = -(\mu_p - 1.64\sigma_p)$,

where $\sigma_p = \sqrt{v'\Sigma v}$ is the standard deviation of the portfolio return.

The figure below illustrates that the optimal portfolio is on the CLM (when the returns are normally distributed)

In [18]:
μ = [9; 6]/100                     #means
Σ = [ 256  0;
      0    144]/10000
Rf = 1/100

mustar  = linspace(Rf,0.1,101)
L       = length(mustar)
StdRp   = fill(NaN,L)
StdRpRf = fill(NaN,L)
for i = 1:L
    StdRp[i]   = MVCalc(mustar[i],μ,Σ)[1]
    StdRpRf[i] = MVCalcRf(mustar[i],μ,Σ,Rf)[1]
end   

VaRRestr = -0.1 + 1.64*StdRpRf

println("let's plot it")

let's plot it


In [19]:
plot(StdRpRf*100,mustar*100,color=:blue,linewidth=2,label="CML",xlims=(0,15),ylims=(0,15))
plot!(StdRp*100,mustar*100,color=:red,linewidth=2,label="MVF")
plot!(StdRpRf*100,VaRRestr*100,line=(:dot,2),color=:green,label="VaR restriction")
title!("Mean vs standard deviation (with Rf)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean vs standard deviation (with Rf) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" points="
 44.6753,338.161 49.6132,336.189 54.5511,334.218 59.489,332.247 64.4268,330.275 69.3647,328.304 74.3026,326.332 79.2405,324.361 84.1783,322.389 89.1162,320.418 
 94.0541,318.447 98.992,316.475 103.93,314.504 108.868,312.532 113.806,310.561 118.743,308.59 123.681,306.618 128.619,304.647 133.557,302.675 138.495,300.704 
 143.433,298.732 148.371,296.761 153.309,294.79 158.246,292.818 163.184,290.847 168.122,288.875 173.06,286.904 177.998,284.933 182.936,282.961 187.874,280.99 
 192.812,279.018 197.75,277.047 202.687,275.075 207.625,273.104 212.563,271.133 217.501,269.161 222.439,267.19 227.377,265.218 232.315,263.247 237.253,261.276 
 242.19,259.304 247.128,257.333 252.066,255.361 257.004,253.39 261.942,251.418 266.88,249.447 271.818,247.476 276.756,245.504 281.693,243.533 286.631,241.561 
 291.569,239.59 296.507,237.619 301.445,235.647 306.383,233.676 311.321,231.704 316.259,229.733 321.196,227.761 326.134,225.79 331.072,223.819 336.01,221.847 
 340.948,219.876 345.886,217.904 350.824,215.933 355.762,213.962 360.699,211.99 365.637,210.019 370.575,208.047 375.513,206.076 380.451,204.104 385.389,202.133 
 390.327,200.162 395.265,198.19 400.202,196.219 405.14,194.247 410.078,192.276 415.016,190.305 419.954,188.333 424.892,186.362 429.83,184.39 434.768,182.419 
 439.705,180.448 444.643,178.476 449.581,176.505 454.519,174.533 459.457,172.562 464.395,170.59 469.333,168.619 474.271,166.648 479.209,164.676 484.146,162.705 
 489.084,160.733 494.022,158.762 498.96,156.791 503.898,154.819 508.836,152.848 513.774,150.876 518.712,148.905 523.649,146.933 528.587,144.962 533.525,142.991 
 538.463,141.019 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 1532.13,338.161 1511.29,336.189 1490.47,334.218 1469.67,332.247 1448.88,330.275 1428.11,328.304 1407.36,326.332 1386.64,324.361 1365.93,322.389 1345.25,320.418 
 1324.6,318.447 1303.97,316.475 1283.37,314.504 1262.79,312.532 1242.25,310.561 1221.74,308.59 1201.26,306.618 1180.81,304.647 1160.4,302.675 1140.03,300.704 
 1119.7,298.732 1099.42,296.761 1079.18,294.79 1058.99,292.818 1038.85,290.847 1018.77,288.875 998.743,286.904 978.78,284.933 958.883,282.961 939.057,280.99 
 919.306,279.018 899.636,277.047 880.053,275.075 860.562,273.104 841.17,271.133 821.885,269.161 802.715,267.19 783.669,265.218 764.757,263.247 745.989,261.276 
 727.378,259.304 708.937,257.333 690.679,255.361 672.622,253.39 654.783,251.418 637.182,249.447 619.84,247.476 602.782,245.504 586.035,243.533 569.628,241.561 
 553.594,239.59 537.97,237.619 522.795,235.647 508.114,233.676 493.976,231.704 480.433,229.733 467.542,227.761 455.365,225.79 443.967,223.819 433.417,221.847 
 423.785,219.876 415.143,217.904 407.562,215.933 401.109,213.962 395.847,211.99 391.83,210.019 389.101,208.047 387.691,206.076 387.617,204.104 388.879,202.133 
 391.462,200.162 395.338,198.19 400.465,196.219 406.788,194.247 414.247,192.276 422.775,190.305 432.3,188.333 442.752,186.362 454.059,184.39 466.153,182.419 
 478.967,180.448 492.441,178.476 506.516,176.505 521.139,174.533 536.261,172.562 551.837,170.59 567.827,168.619 584.195,166.648 600.906,164.676 617.931,162.705 
 635.242,160.733 652.816,158.762 670.629,156.791 688.663,154.819 706.899,152.848 725.321,150.876 743.913,148.905 762.664,146.933 781.561,144.962 800.593,142.991 
 819.749,141.019 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 44.6753,579.112 49.6132,574.144 54.5511,569.177 59.489,564.209 64.4268,559.242 69.3647,554.274 74.3026,549.307 79.2405,544.339 84.1783,539.372 8